In [185]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# Saving the Goat

An agricultural economist has to sell a *wolf*, a *goat*, and a *cabbage* on a market place.  In order to
reach the market place, she has to cross a river.  The boat that she can use is so small that it can
only accommodate either the goat, the wolf, or the cabbage in addition to the agricultural economist herself.
Now if the agricultural economist leaves the wolf alone with the goat, the wolf will eat the goat.
If, instead, the agricultural economist leaves the goat with the cabbage, the goat will eat the cabbage.
Is it possible for the agricultural economist to develop a schedule that allows her to cross the river
without either the goat or the cabbage being eaten?

You can try to solve the  yourself at: [http://www.mathcats.com/explore/river/crossing.html](http://www.mathcats.com/explore/river/crossing.html)


We will encode this problem as a *symbolic transition system* and then solve it with the help of our *constraint solver*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{F}i$ for $i\in\{0,\cdots,n\}$ is the number of farmers on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{W}i$ for $i\in\{0,\cdots,n\}$ is the number of wolves on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{G}i$ for $i\in\{0,\cdots,n\}$ is the number of goats on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of cabbages on the western shore after the 
  $i^{\textrm{th}}$ crossing.

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [186]:
def flatten(LoL):
    return [x for L in LoL for x in L]

In [187]:
flatten([[1,2], [3,4]])

[1, 2, 3, 4]

The function `start` takes four natural numbers as input:
* `F` is the number of farmers on the western shore,
* `W` is the number of wolves on the western shore,
* `G` is the number of goats on the western shore, and
* `C` is the number of cabbages on the western shore,

It returns `True` in the initial state where everybody is on the western shore.

In [188]:
def start(F, W, G, C):
    return (F == 1 and W == 1 and G == 1 and C == 1)

The function `goal` takes four natural numbers as input:
* `F` is the number of farmers on the western shore,
* `W` is the number of wolves on the western shore,
* `G` is the number of goats on the western shore, and
* `C` is the number of cabbages on the western shore.

It returns `True` in the state where everybody is on the eastern shore and hence nobody is on the western shore.

In [189]:
def goal(F, W, C, G):
    return (F == 0 and W == 0 and G == 0 and C == 0)

The function `implies(A, B)` takes two Boolean values as its input.  It returns the Boolean value
$$ A \rightarrow B. $$

In [190]:
def implies(A, B):
    return not A or B

The function `invariant` takes four natural numbers as input:
* `F` is the number of farmers on the western shore,
* `W` is the number of wolves on the western shore,
* `G` is the number of goats on the western shore, and
* `C` is the number of cabbages on the western shore.

It returns `True` if there is no problem on either shore of the river.  There is no problem if on the shore where there is no farmer, the following conditions hold:
* If the wolf is on that shore, the goat must be on the opposite shore.
* If the goat is on that shore, the cabbage has to be on the opposite shore.

In [233]:
def invariant(F, W, G, C):
    return (F == 1 or (W != 1 or G != 1) and (G != 1 or C != 1) and
            F == 0 or (W != 0 or G != 0) and (G != 0 or C != 0))

The function `transition` takes 8 arguments:
* `F𝛼` is the number of farmers on the western shore before the crossing.
* `W𝛼` is the number of wolves on the western shore before the crossing.
* `G𝛼` is the number of goats on the western shore before the crossing. 
* `C𝛼` is the number of cabbages on the western shore before the crossing. 
* `F𝛽` is the number of farmers on the western shore after the crossing.
* `W𝛽` is the number of wolves on the western shore after the crossing.
* `G𝛽` is the number of goats on the western shore after the crossing.
* `C𝛽` is the number of cabbages on the western shore after the crossing.

It returns a `True` if the crossing is admissible, i.e.:
* The farmer travels in every crossing.
* She takes at most one of her goods with her in the boat.
* If the farmer travels from the western shore to the eastern shore,
  the number of goods on the western shore stays the same or decreases, otherwise it stays the same or increases. 

In [234]:
def transition(F𝛼, W𝛼, G𝛼, C𝛼, F𝛽, W𝛽, G𝛽, C𝛽):
    return (abs(F𝛼 - F𝛽) == 1 
            and implies(F𝛼 == 1, (0 <= ((W𝛼 - W𝛽) + (G𝛼 - G𝛽) + (C𝛼 - C𝛽)) <= 1) and W𝛽 <= W𝛼 and G𝛽 <= G𝛼 and C𝛽 <= C𝛼 )
            and implies(F𝛼 == 0, (0 <= ((W𝛽 - W𝛼) + (G𝛽 - G𝛼) + (C𝛽 - C𝛼)) <= 1) and W𝛼 <= W𝛽 and G𝛼 <= G𝛽 and C𝛼 <= C𝛽 ))

The function `wgc_CSP` creates a CSP that tries to solve the problem with `n` crossings.

In [235]:
def wgc_CSP(n):
    Lists = [[f'F{i}', f'W{i}', f'G{i}', f'C{i}'] for i in range(n+1)]
    Variables = flatten(Lists)
    Values = {0, 1}
    
    Constraints = set()
    Constraints |= {'F0 == 1', 'W0 == 1', 'G0 == 1', 'C0 == 1'}
    Constraints |= {f'F{n} == 0', f'W{n} == 0', f'G{n} == 0', f'C{n} == 0'} 

    for i in range(n+1):
        Constraints |= {f'invariant(F{i}, W{i}, G{i}, C{i})'}
    for i in range(n):
        Constraints |= {f'transition(F{i}, W{i}, G{i}, C{i}, F{i+1}, W{i+1}, G{i+1}, C{i+1})'}

    return Variables, Values, Constraints

In [236]:
wgc_CSP(3)

(['F0',
  'W0',
  'G0',
  'C0',
  'F1',
  'W1',
  'G1',
  'C1',
  'F2',
  'W2',
  'G2',
  'C2',
  'F3',
  'W3',
  'G3',
  'C3'],
 {0, 1},
 {'C0 == 1',
  'C3 == 0',
  'F0 == 1',
  'F3 == 0',
  'G0 == 1',
  'G3 == 0',
  'W0 == 1',
  'W3 == 0',
  'invariant(F0, W0, G0, C0)',
  'invariant(F1, W1, G1, C1)',
  'invariant(F2, W2, G2, C2)',
  'invariant(F3, W3, G3, C3)',
  'transition(F0, W0, G0, C0, F1, W1, G1, C1)',
  'transition(F1, W1, G1, C1, F2, W2, G2, C2)',
  'transition(F2, W2, G2, C2, F3, W3, G3, C3)'})

In [237]:
%run 02-Backtracking-Constraint-Solver.ipynb

The function `find_solution` computes a solution to the *wolf-goat-cabbage* problem.

In [238]:
def find_solution():
    n = 1
    while True:
        print(n)
        CSP = wgc_CSP(n)
        Solution = solve(CSP)
        if Solution != None:
            return n, Solution
        n += 2

Solving the problem should take less than a second.

In [239]:
%%time
n, Solution = find_solution()
n, Solution

1
3
5
CPU times: total: 15.6 ms
Wall time: 23.7 ms


(5,
 {'F0': 1,
  'W0': 1,
  'G0': 1,
  'C0': 1,
  'F1': 0,
  'W1': 0,
  'G1': 1,
  'C1': 1,
  'F2': 1,
  'W2': 0,
  'G2': 1,
  'C2': 1,
  'F3': 0,
  'W3': 0,
  'G3': 0,
  'C3': 1,
  'F4': 1,
  'W4': 0,
  'G4': 0,
  'C4': 1,
  'F5': 0,
  'W5': 0,
  'G5': 0,
  'C5': 0})

In [214]:
def show_solution(Solution, n):
    for i in range(n+1):
        F = Solution[f'F{i}']
        W = Solution[f'W{i}']
        G = Solution[f'G{i}']
        C = Solution[f'C{i}']
        print('🧑‍🌾'*F + '🐺'*W + '🐐'*G + '🥦'*C + ' ' * 28 + '🧑‍🌾'*(1-F) + '🐺'*(1-W) + '🐐'*(1-G) + '🥦'*(1-C))
        if F == 1:
            WB = Solution[f'W{i}'] - Solution[f'W{i+1}']
            GB = Solution[f'G{i}'] - Solution[f'G{i+1}']
            CB = Solution[f'C{i}'] - Solution[f'C{i+1}']
            print(' ' * 12 + '>>> 🧑‍🌾' + '🐺'*WB + '🐐'*GB + '🥦'*CB + ' >>>')
        elif i + 1 < n:
            WB = Solution[f'W{i+1}'] - Solution[f'W{i}']
            GB = Solution[f'G{i+1}'] - Solution[f'G{i}']
            CB = Solution[f'C{i+1}'] - Solution[f'C{i}']
            print(' ' * 12 + '<<< 🧑‍🌾' + '🐺'*WB + '🐐'*GB + '🥦'*CB + ' <<<')

In [215]:
show_solution(Solution, n)

🧑‍🌾🐺🐐🥦                            
            >>> 🧑‍🌾🐐 >>>
🐺🥦                            🧑‍🌾🐐
            <<< 🧑‍🌾 <<<
🧑‍🌾🐺🥦                            🐐
            >>> 🧑‍🌾🐺 >>>
🥦                            🧑‍🌾🐺🐐
            <<< 🧑‍🌾🐐 <<<
🧑‍🌾🐐🥦                            🐺
            >>> 🧑‍🌾🥦 >>>
🐐                            🧑‍🌾🐺🥦
            <<< 🧑‍🌾 <<<
🧑‍🌾🐐                            🐺🥦
            >>> 🧑‍🌾🐐 >>>
                            🧑‍🌾🐺🐐🥦
